# Modelo processo linguagem natural - Tweets

In [ ]:
# Versão do python
from platform import python_version

print('Versão Jupyter Notebook neste projeto:', python_version())

In [ ]:
#Importação das bibliotecas

# Bibliotecas para NLTK
import nltk
import re
import wordcloud
import itertools
from wordcloud import WordCloud

import pandas as pd # Carregamento de arquivos de csv
import numpy as np # Carregamento cálculos em arrays multidimensionais

# Bibliotecas de visualização
import seaborn as sns
import matplotlib.pyplot as plt

# Carregar as versões das bibliotecas
import watermark

# Warnings retirar alertas 
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Baixando pacote do punkt

import nltk
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
# Verficações da versões das bibliotecas

%reload_ext watermark
%watermark -a "Rafael Gallo" --iversions

In [ ]:
# Configuração fundo dos gráficos e estilo, tamanho da fonte

sns.set_palette("Accent")
sns.set(style="whitegrid", color_codes=True, font_scale=1.3)
color = sns.color_palette()

# Base dados

In [ ]:
df = pd.read_csv("Tweets.csv")
df

In [ ]:
# Exebindo o 5 primeiro dados 
df.head(5)

In [ ]:
# Exebindo o 5 últimos dados
df.tail(5)

In [ ]:
# Número de linhas e colunas 
df.shape

In [ ]:
# Exibido os tipos de dados
df.dtypes

In [ ]:
# Informando as informações e das variaveis 
df.info()

In [ ]:
# Total de colunas e linhas 

print("Rows:", df.shape[0])
print("Columns:", df.shape[1])

In [ ]:
# Exibindo valores ausentes e Valores únicos

print("\nMissing values :  ", df.isnull().sum().values.sum())
print("\nUnique values :  \n",df.nunique())

In [ ]:
# Total de número duplicados
df.duplicated()

In [ ]:
# Sentimento do coluna 
df.sentiment

In [ ]:
# Contando números de dados
df.sentiment.value_counts()

In [ ]:
# Renomeando as colunas do dataset

df.columns = ["Id",
              "Texto",
              "Sub_Texto",
              "Sentimento"]
df.head()

In [ ]:
# Contagem de dados da coluna na Sentimento
df.Sentimento.count()

In [ ]:
# Contagem de dados da coluna na Sub_Texto
df.Sub_Texto.count()

In [ ]:
# Textos duplicados total

df.drop_duplicates(["Texto"], inplace = True)
df.Texto.count()

# Análise de dados

In [ ]:
# Gráfico barras de sentimento
plt.figure(figsize=(12.8,6))

ax = sns.countplot(df["Sentimento"])
plt.title("Análise de sentimento")
plt.xlabel("Sentimentos")
plt.ylabel("Total de sentimentos")
plt.show()

# Treino teste
- Treino e teste da base de dados da colunas textos e sentimento

In [ ]:
train = df["Texto"] # Variável para treino
test = df["Sentimento"] # Variável para teste

In [ ]:
# Total de linhas e colunas dados variável x
train.shape

In [ ]:
# Total de linhas e colunas dados variável y
test.shape

# Pré-processamento

In [ ]:
# Dados de limpeza para modelo PLN

from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

# Remove stop words: Removendo as stop words na base de dados
def remove_stop_words(instancia): # Removendo as stop words
    stopwords = set(nltk.corpus.stopwords.words("english"))
    palavras = [i for i in instancia.split() if not i in stopwords]
    return (" ".join(palavras))

# Palavras derivacionalmente relacionadas com significados semelhantes, palavras para retornar documentos que contenham outra palavra no conjunto.
def text_stemming(instancia):
    stemmer = nltk.stem.RSLPStemmer()
    palavras = []
    for w in instancia.split():
        palavras.append(stemmer.stem(w))
        return (" ".join(palavras))

# Limpeza na base de dados limpando dados de web com http e outros.
def dados_limpos(instancia): 
    instancia = re.sub(r"http\S+", "", instancia).lower().replace('.','').replace(';','').replace('-','').replace(':','').replace(')','')
    return (instancia)

#Lemmatization: Em linguística é o processo de agrupar as formas flexionadas de uma palavra para que possam ser analisadas como um único item, identificado pelo lema da palavra , ou forma de dicionário.
def Lemmatization(instancia):
    palavras = []
    for w in instancia.split():
        palavras.append(wordnet_lemmatizer.lemmatize(w))
        return (" ".join(palavras))

# Preprocessing: Pré - processamento da base de dados que serão ser para análise de dados.
def Preprocessing(instancia):
    instancia = re.sub(r"http\S+", "", instancia).lower().replace('.','').replace(';','').replace('-','').replace(':','').replace(')','').replace('"','')
    stopwords = set(nltk.corpus.stopwords.words('english'))
    palavras = [i for i in instancia.split() if not i in stopwords]
    return (" ".join(palavras))

letters_only = re.sub("[^a-zA-Z]",  # Search for all non-letters
                          " ",          # Replace all non-letters with spaces
                          str(location))

# Função para texto de negações
def marque_negacao(texto):
    
    # Negaçoes do texto mudando para not para "não"
    negacoes = ['não','not']
    negacao_detectada = False
    
    # Criando uma lista vazia 
    resultado = []
    palavras = texto.split()
    
    # For em palavras para os dados de negações 
    for p in palavras:
        p = p.lower()
        if negacao_detectada == True:
            p = p + '_NEG'
        if p in negacoes:
            negacao_detectada = True
        resultado.append(p)
    
    # Retornando a função
    return (" ".join(resultado))

In [ ]:
# Base dados limpo
train = [Preprocessing(i) for i in train]
train[:50]

In [ ]:
# Tokenização as palavras precisam ser codificadas como inteiros, 
# Ou valores de ponto flutuante, para serem usadas como entradas para modelos machine learning.
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer

tokenizer = TweetTokenizer()
vet = CountVectorizer(analyzer="word", tokenizer = tokenizer.tokenize)
vet_train = vet.fit_transform(train)
vet_train.shape